In [12]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model 
from PIL import Image
import os

In [2]:
try:
  # colab
  %tensorflow_version 2.x
except Exception:
    pass

IMG_SHAPE = (128, 128, 3)

base_model = tf.keras.applications.MobileNet(input_shape=IMG_SHAPE, include_top=False)
base_model.trainable = False

model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(3, activation='softmax')
])

model.summary()

model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_128 (Function (None, 4, 4, 1024)        3228864   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 3)                 3075      
Total params: 3,231,939
Trainable params: 3,075
Non-trainable params: 3,228,864
_________________________________________________________________


In [3]:
zip_file = tf.keras.utils.get_file(origin="./Users/ProFik/.keras/datasets/data",
                                   fname="data.zip", extract=True)
base_dir, _ = os.path.splitext(zip_file)

In [9]:
train_dir = os.path.join(base_dir, 'training_set')
validation_dir = os.path.join(base_dir, 'test_set')
image_size = 224
batch_size = 32

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(directory=train_dir, target_size=(image_size, image_size), batch_size=batch_size)

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

validation_generator = validation_datagen.flow_from_directory(directory=validation_dir, target_size=(image_size, image_size), batch_size=batch_size)

IMG_SHAPE = (image_size, image_size, 3)

base_model = tf.keras.applications.MobileNet(input_shape=IMG_SHAPE, include_top=False)
base_model.trainable = False

Found 5668 images belonging to 3 classes.
Found 2665 images belonging to 3 classes.
17227776/17225924 [==============================] - 2s 0us/step


In [5]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(3, activation='softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_128 (Function (None, 4, 4, 1024)        3228864   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 3075      
Total params: 3,231,939
Trainable params: 3,075
Non-trainable params: 3,228,864
_________________________________________________________________


In [11]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

epochs = 33
steps_per_epoch = np.ceil(train_generator.n / batch_size)
validation_steps = np.ceil(validation_generator.n / batch_size)

history = model.fit_generator(generator=train_generator,
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs,
                              validation_data=validation_generator,
                              validation_steps=validation_steps)

model.save("model.h5")

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
  5/178 [..............................] - ETA: 1:09 - loss: 1.2366 - accuracy: 0.4437

KeyboardInterrupt: 

# Testing on own images

In [20]:
img_size = 224

img = Image.open(r'C:\Users\ProFik\python_projects\mydata\main_set\cats\6.jpg')
img = img.convert('RGB')
img = img.resize((img_size, img_size), Image.ANTIALIAS)
img = np.array(img).reshape(-1, img_size, img_size, 3)

#model = load_model("model.h5")

result = model.predict(img)

for i in range(len(result)):
    result[i] *= 100
    
result = np.around(result, decimals = 1)

print(result)

[[99.8  0.   0.2]]


In [21]:
print('Киса с вероятностью ', result[0][0], end='%')

Киса с вероятностью  99.8%

In [ ]:
saved_model_dir = '/content/TFLite'
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
    f.write(labels)